In [1]:
# Auto PEP8: pip install nb_black
# Loading extension
%load_ext nb_black

<IPython.core.display.Javascript object>

In [7]:
import plotly.graph_objects as go
import pandas as pd

<IPython.core.display.Javascript object>

In [8]:
animals = ["giraffes", "orangutans", "monkeys"]

fig = go.Figure(
    data=[
        go.Bar(name="Job 1", x=[1, 1, 1], y=[0, 0, 0], base=[1, 3, 5], orientation="h"),
        go.Bar(name="Job 2", x=[0, 3, 1], y=[1, 1, 1], base=[1, 2, 5], orientation="h"),
    ]
)
# Change the bar mode
fig.show()

<IPython.core.display.Javascript object>

In [9]:
from typing import Union
# Create mock dataset for testing
# Mock Inputs (refer to slides - it follows the same format as the data given there)
# Start from task 0 though ;) 
task_1 = [0, 3, 8, 2, 1]
task_2 = [1, 3, 10, 1, 1]
task_3 = [2, 1, 14, 1, 1]
tasks = [task_1, task_2, task_3]

# Dash table outputs its property "data" for every row, where key is the column, and value is the value
# Left spaces in the names since they are used for the column names in Dash Table
task_template = ["Task", "Worst Case", "Period", "Invocation-1", "Invocation-2"]

table_data = []

for task in tasks:
    task_tup = tuple(zip(task_template, task))
    task_dict = {key: val for key, val in task_tup}
    table_data.append(task_dict)
    
# Dash table outputs its property "column" for every column name, where key is the column_id, and value is the value
# Sadly we need to iterate over the "data" property to get the Invocation values
table_col = [{key: key} for key in task_template]

def generate_init_data() -> Union[list, list]:

    task_1 = [0, 3, 8, 2, 1]
    task_2 = [1, 3, 10, 1, 1]
    task_3 = [2, 1, 14, 1, 1]
    tasks = [task_1, task_2, task_3]

    task_template = ["Task", "Worst Case", "Period", "Invocation-1", "Invocation-2"]

    table_data = []

    for task in tasks:
        task_tup = tuple(zip(task_template, task))
        task_dict = {key: val for key, val in task_tup}
        table_data.append(task_dict)
    table_col = [{"name": val, "id": val, 'type': 'numeric',} for val in task_template]
    return table_data, table_col

# Dash table outputs its property "column" for every column name, where key is the column_id, and value is the value
# Sadly we need to iterate over the "data" property to get the Invocation values
table_col = [{"name": val, "id": val, 'type': 'numeric'} for val in task_template]
print("Table Data: \n", table_data)
print("Table Columns: \n", table_col)


Table Data: 
 [{'Task': 0, 'Worst Case': 3, 'Period': 8, 'Invocation-1': 2, 'Invocation-2': 1}, {'Task': 1, 'Worst Case': 3, 'Period': 10, 'Invocation-1': 1, 'Invocation-2': 1}, {'Task': 2, 'Worst Case': 1, 'Period': 14, 'Invocation-1': 1, 'Invocation-2': 1}]
Table Columns: 
 [{'name': 'Task', 'id': 'Task', 'type': 'numeric'}, {'name': 'Worst Case', 'id': 'Worst Case', 'type': 'numeric'}, {'name': 'Period', 'id': 'Period', 'type': 'numeric'}, {'name': 'Invocation-1', 'id': 'Invocation-1', 'type': 'numeric'}, {'name': 'Invocation-2', 'id': 'Invocation-2', 'type': 'numeric'}]


<IPython.core.display.Javascript object>

In [10]:
# Data parsing to get inputs we need for our algorithim
df_params = pd.DataFrame(table_data)

# Invocations to list
df_invoc = df_params.filter(regex="Invocation")
task_invoc = [df_invoc[col].tolist() for col in df_invoc.columns]
print("df_params: \n", df_params)
print("df_invoc: \n", df_invoc)

df_params: 
    Task  Worst Case  Period  Invocation-1  Invocation-2
0     0           3       8             2             1
1     1           3      10             1             1
2     2           1      14             1             1
df_invoc: 
    Invocation-1  Invocation-2
0             2             1
1             1             1
2             1             1


<IPython.core.display.Javascript object>

In [11]:
import pandas as pd

# Stop at missed deadline
def edf_algo(df_params: pd.DataFrame, fm_all=False, fm_val=1) -> list:
    deadline = False
    curr_period = 0
    start_time = []
    end_time = []

    # Get lists from Pandas (way easier than converting Table Data mess)
    # There is probably a better way to do this...
    task_id = df_params["Task"].tolist()
    task_state = df_params["Worst Case"].tolist()
    task_wc = df_params["Worst Case"].tolist()
    task_period = df_params["Period"].tolist()

    # Plotly data templating
    plot_data = []
    hovertemplate = (
        "<b>Start:</b> %{base:.2f}s<br>"
        + "<b>Finish:</b> %{x:.2f}s<br>"
        + "<b>%{text}</b>"
    )

    for task in task_id:
        fig = {
            "name": "Task-{}".format(task),
            "x": [],
            "y": [],
            "base": [],
            "text": [],
            "type": "bar",
            "orientation": "h",
            "hovertemplate": hovertemplate,
        }
        plot_data.append(fig)

    # Get a list of invocations
    df_invoc = df_params.filter(regex="Invocation")
    task_invoc = [df_invoc[col].tolist() for col in df_invoc.columns]

    for inv_num, invocation in enumerate(task_invoc, start=1):
        for task_num in range(len(task_state)):
            # If at start of algo
            if inv_num == 1 and task_num == 0:
                t_start = 0
            # If waiting till next period
            elif inv_num is not 1 and start_time[-1] < (
                curr_period * task_period[task_num]
            ):
                t_start = curr_period * task_period[task_num]
            # Else make last end time start time (sequential)
            else:
                t_start = end_time[-1]

            start_time.append(t_start)

            # Set running task T as worst case
            task_state[task_num] = task_wc[task_num]

            # Calculate utility
            util = [
                task_state[idx] / task_period[idx] for idx in range(len(task_state))
            ]
            util = sum(util) * fm_val

            # Round to other util if enabled
            if fm_all:
                if util < 0.5:
                    util = 0.5
                elif util < 0.75:
                    util = 0.75

            # Set task state to invocation number
            task_state[task_num] = invocation[task_num]

            # Calculate t
            t = invocation[task_num] / util

            # If waiting till next period
            if inv_num is not 1 and start_time[-1] < (
                curr_period * task_period[task_num]
            ):
                t_end = curr_period * task_period[task_num] + t
            #                 end_time.append(t_end)
            else:
                t_end = t + start_time[-1]

            end_time.append(t_end)

            # Append data to plotly figures
            fig = plot_data[task_num]

            task_number = task_id[task_num]
            fig["x"].append(t_end - t_start)
            fig["y"].append("Task-{}".format(task_number))
            fig["text"].append("Frequency (Fm={}): {:.3f}".format(fm_val, util))
            fig["base"].append(t_start)

            # Hacky way to check if we past the deadline (the two breaks irk me :()
            if t > (inv_num * task_period[task_num]):
                deadline = True
                print("Calculated deadline >>> ", t)
                print("Period >>> {}".format(inv_num * task_period[task_num]))
                break

        if deadline:
            break
        curr_period = +1

    print("start time: >>>", start_time)
    print("end time: >>>", end_time)
    plot_data = [go.Bar(fig) for fig in plot_data]
    return plot_data


data = edf_algo(df_params)

start time: >>> [0, 2.679425837320574, 4.288621239619425, 8, 10, 14]
end time: >>> [2.679425837320574, 4.288621239619425, 6.661502595551628, 9.830065359477125, 12.014388489208633, 17.373493975903614]


<IPython.core.display.Javascript object>

In [12]:
# Render graph
fig = go.Figure(data=data)
fig.update_layout(
    title="EDF Graph",
    xaxis_title="Time (s)",
    yaxis_title="Task",
)
fig.show()

<IPython.core.display.Javascript object>

In [13]:
# TODO (Algo):
# 1. Test with multiple frequencies
# 2. Test more than 2 invocations, in specific the following cases
# 2.a Case where invocations all end early after 1st invocation
# 2.b Case where invocations all start at their period (seems to work ATM)
# Add fm_val functionality (I think this works) - basically just need to test above

# TODO (Dashboard) - I can help
# 1. Hook up multiple frequencies with callback
# 2. Add line indicating that we missed a deadline or print it out
# 3. Edge cases. User enters multiple tasks with same task number. Limiting user input to range, etc.

# TODO: Questions?
# Does this algorithim run items based on task or period first?
# If it's period first than we need to used Pandas to reorder the dataset in ascending order based on period. Really easy :)

<IPython.core.display.Javascript object>

In [14]:
import dash_table as dt
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import pandas as pd
import plotly.graph_objects as go
from jupyter_dash import JupyterDash


# Build App
table_data, table_cols = generate_init_data()
print(table_cols)
app = JupyterDash(__name__)
app.layout = html.Div(
    [
        # Header
        html.Div(
            [
                html.H1("ENGR 451 - Cycle Conserving EDF Algorithim"),
            ]
        ),
        html.Div(
            [
                html.Div(
                    [
                        dt.DataTable(
                            id="fm-table",
                            columns=table_cols,
                            data=table_data,
                            editable=True,
                            row_deletable=True,
                        )
                    ],
                    style={"margin": "1.5% 0%"},
                    className="row",
                ),
                html.Div(
                    [
                        html.Div(
                            [
                                html.Label("Input Fm"),
                                dcc.Input(
                                    id="fm-input",
                                    type="number",
                                    value=1,
                                    min=1,
                                    max=10,
                                    step=1,
                                    style={"width": "100%"},
                                ),
                            ],
                            className="two columns",
                        ),
                        html.Div(
                            [
                                html.Label("Select Fm"),
                                dcc.Dropdown(
                                    id="fm-dropdown",
                                    options=[
                                        {
                                            "label": "Fm (All Frequencies)",
                                            "value": 0,
                                        },
                                        {
                                            "label": "Fm, 0.75Fm, & 0.5Fm",
                                            "value": 1,
                                        },
                                    ],
                                    value=0,
                                ),
                            ],
                            className="three columns",
                        ),
                        html.Div(
                            [
                                html.Br(),
                                html.Button(
                                    "Add Invocation",
                                    id="invocation-button",
                                    n_clicks=0,
                                    style={"width": "100%"},
                                ),
                            ],
                            className="three columns",
                        ),
                        html.Div(
                            [
                                html.Br(),
                                html.Button(
                                    "Add Task",
                                    id="rows-button",
                                    n_clicks=0,
                                    style={"width": "100%"},
                                ),
                            ],
                            className="two columns",
                        ),
                        html.Div(
                            [
                                html.Br(),
                                html.Button(
                                    "Run",
                                    id="run-button",
                                    style={"float": "right"},
                                ),
                            ],
                            className="two columns",
                        ),
                    ],
                    style={"margin": "1.5% 0%"},
                    className="row",
                ),
            ],
            className="row",
        ),
        html.Div(
            [
                dcc.Graph(id="fm-graph"),
            ],
        ),
        dcc.Store(id="invocation-store", data=2),
    ],
    style={"margin": "0% 1%"},
)

# Update graph with algorithim
@app.callback(
    Output("fm-graph", "figure"),
    [Input("run-button", "n_clicks")],
    [
        State("fm-table", "columns"),
        State("fm-table", "data"),
        State("fm-input", "value"),
        State("fm-dropdown", "value"),
    ],
)
def update_figure(run_button, table_cols, table_data, fm_val, fm_all):
    print(table_data)
    df_params = pd.DataFrame(table_data)
    print(df_params)
    data = edf_algo(df_params, fm_all=fm_all, fm_val=fm_val)
    fig = go.Figure(data=data)
    fig.update_layout(
        title="EDF Graph",
        xaxis_title="Time (s)",
        yaxis_title="Task",
    )
    return fig


# Add invocation table column
@app.callback(
    [
        Output("invocation-store", "data"),
        Output("fm-table", "columns"),
    ],
    [Input("invocation-button", "n_clicks")],
    [
        State("fm-table", "columns"),
        State("fm-table", "data"),
        State("invocation-store", "data"),
    ],
)
def add_table_col(invocation_button, table_cols, table_data, invoc_num):
    if invocation_button > 0:
        invoc_num += 1
        invoc_name = "Invocation-{}".format(invoc_num)
        table_cols.append(
            {"id": invoc_name, "name": invoc_name, "deletable": True, "type": "numeric"}
        )

    return invoc_num, table_cols


# Add rows to table
@app.callback(
    Output("fm-table", "data"),
    [Input("rows-button", "n_clicks")],
    [State("fm-table", "data"), State("fm-table", "columns")],
)
def add_rows(rows_button, table_rows, table_cols):
    if rows_button > 0:
        table_rows.append({c["id"]: "" for c in table_cols})
    return table_rows


# Run app and display result inline in the notebook
app.run_server(mode="external")

[{'name': 'Task', 'id': 'Task', 'type': 'numeric'}, {'name': 'Worst Case', 'id': 'Worst Case', 'type': 'numeric'}, {'name': 'Period', 'id': 'Period', 'type': 'numeric'}, {'name': 'Invocation-1', 'id': 'Invocation-1', 'type': 'numeric'}, {'name': 'Invocation-2', 'id': 'Invocation-2', 'type': 'numeric'}]
[{'Task': 0, 'Worst Case': 3, 'Period': 8, 'Invocation-1': 2, 'Invocation-2': 1}, {'Task': 1, 'Worst Case': 3, 'Period': 10, 'Invocation-1': 1, 'Invocation-2': 1}, {'Task': 2, 'Worst Case': 1, 'Period': 14, 'Invocation-1': 1, 'Invocation-2': 1}]
   Task  Worst Case  Period  Invocation-1  Invocation-2
0     0           3       8             2             1
1     1           3      10             1             1
2     2           1      14             1             1
start time: >>> [0, 2.679425837320574, 4.288621239619425, 8, 10, 14]
end time: >>> [2.679425837320574, 4.288621239619425, 6.661502595551628, 9.830065359477125, 12.014388489208633, 17.373493975903614]
Dash app running on http:/

<IPython.core.display.Javascript object>

[{'Task': 0, 'Worst Case': 3, 'Period': 8, 'Invocation-1': 2, 'Invocation-2': 1}, {'Task': 1, 'Worst Case': 3, 'Period': 10, 'Invocation-1': 1, 'Invocation-2': 1}, {'Task': 2, 'Worst Case': 1, 'Period': 14, 'Invocation-1': 1, 'Invocation-2': 1}]
   Task  Worst Case  Period  Invocation-1  Invocation-2
0     0           3       8             2             1
1     1           3      10             1             1
2     2           1      14             1             1
start time: >>> [0, 2.6666666666666665, 4.0, 8, 10, 14]
end time: >>> [2.6666666666666665, 4.0, 6.0, 9.333333333333334, 12.0, 16.0]
[{'Task': 0, 'Worst Case': 3, 'Period': 8, 'Invocation-1': 2, 'Invocation-2': 1}, {'Task': 1, 'Worst Case': 3, 'Period': 10, 'Invocation-1': 1, 'Invocation-2': 1}, {'Task': 2, 'Worst Case': 1, 'Period': 14, 'Invocation-1': 1, 'Invocation-2': 1}]
   Task  Worst Case  Period  Invocation-1  Invocation-2
0     0           3       8             2             1
1     1           3      10             